# This notebook is an extension of loan_ml-train_with_investor_features.ipynb just with keras

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('lending_club_ml.csv')
df.drop(['id'], inplace=True, axis=1)

C:\Users\leero\Anaconda3\envs\tf_gpu\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# investor only features
df_lc = df[['loan_status', 'loan_amnt', 'purpose', 'sub_grade', 'int_rate', 'term',  'installment',
            'home_ownership', 'emp_length',  'annual_inc', 'dti',
            'fico_range_low', 'fico_range_high', 'open_acc', 'total_acc', 'revol_bal', 'revol_util', 'inq_last_6mths', 'acc_now_delinq',
            'delinq_amnt', 'delinq_2yrs', 'mths_since_last_delinq', 'pub_rec', 'mths_since_last_record', 'mths_since_last_major_derog', 'collections_12_mths_ex_med']]

In [4]:
# dummy variables
# I have decided to use subgrade and not grade as these are suppose to be similar features but, subgrade is more granular

df_lc = pd.concat([df_lc, pd.get_dummies(
    df_lc.sub_grade, prefix='sub_grade', drop_first=True)], axis=1)
df_lc = pd.concat([df_lc, pd.get_dummies(df_lc.home_ownership,
                                         prefix='home_ownership', drop_first=True)], axis=1)
df_lc = pd.concat([df_lc, pd.get_dummies(
    df_lc.purpose, prefix='purpose', drop_first=True)], axis=1)

df_lc.drop(columns=['sub_grade', 'home_ownership', 'purpose'], inplace=True)

In [5]:
# fill all nans with 0
df_lc.fillna(0, inplace=True)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df_lc.drop(['loan_status'], axis=1)
y = df_lc.loan_status


scaler = StandardScaler()

X_normalized = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

def score(x,y,model):
    print(np.mean(y))
    print(confusion_matrix(y, model.predict_classes(x)))
    print(classification_report(y, model.predict_classes(x)))

#def precision(y_true, y_pred):
#    return tf.keras.backend.precision(y_true, y_pred)


In [8]:
import tensorflow as tf

x_len = len(X.columns)

some_activation = tf.keras.layers.LeakyReLU(alpha=0.3)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(x_len, activation=some_activation, input_shape=(x_len,)))
model.add(tf.keras.layers.Dense(1000, activation=some_activation))
model.add(tf.keras.layers.Dense(2000, activation=some_activation))
model.add(tf.keras.layers.Dense(2000, activation=some_activation))
model.add(tf.keras.layers.Dense(2500, activation=some_activation))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy','categorical_accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
for number in range(11):
    print(number)
    model.fit(Xn_train, yn_train, validation_split=0.25, epochs=5)
    score(Xn_test,yn_test,model)

0
Train on 1316037 samples, validate on 438679 samples
Epoch 1/5
1316037/1316037 [==============================] - 264s 201us/sample - loss: 0.4544 - acc: 0.8028 - categorical_accuracy: 0.0152 - val_loss: 0.4512 - val_acc: 0.8041 - val_categorical_accuracy: 0.0151
Epoch 2/5
1316037/1316037 [==============================] - 280s 213us/sample - loss: 0.4511 - acc: 0.8038 - categorical_accuracy: 0.0241 - val_loss: 0.4505 - val_acc: 0.8042 - val_categorical_accuracy: 0.0169
Epoch 3/5
1316037/1316037 [==============================] - 277s 210us/sample - loss: 0.4502 - acc: 0.8042 - categorical_accuracy: 0.0256 - val_loss: 0.4498 - val_acc: 0.8050 - val_categorical_accuracy: 0.0205
Epoch 4/5
1316037/1316037 [==============================] - 275s 209us/sample - loss: 0.4494 - acc: 0.8045 - categorical_accuracy: 0.0264 - val_loss: 0.4494 - val_acc: 0.8051 - val_categorical_accuracy: 0.0283
Epoch 5/5
1316037/1316037 [==============================] - 271s 206us/sample - loss: 0.4487 - acc: 

1316037/1316037 [==============================] - 265s 201us/sample - loss: 0.4313 - acc: 0.8112 - categorical_accuracy: 0.0492 - val_loss: 0.4480 - val_acc: 0.8038 - val_categorical_accuracy: 0.0581
Epoch 5/5
1316037/1316037 [==============================] - 265s 201us/sample - loss: 0.4302 - acc: 0.8117 - categorical_accuracy: 0.0503 - val_loss: 0.4467 - val_acc: 0.8037 - val_categorical_accuracy: 0.0681
0.8009672589365736
[[ 26359 123318]
 [ 25455 576890]]
              precision    recall  f1-score   support

           0       0.51      0.18      0.26    149677
           1       0.82      0.96      0.89    602345

    accuracy                           0.80    752022
   macro avg       0.67      0.57      0.57    752022
weighted avg       0.76      0.80      0.76    752022

6
Train on 1316037 samples, validate on 438679 samples
Epoch 1/5
1316037/1316037 [==============================] - 267s 203us/sample - loss: 0.4290 - acc: 0.8121 - categorical_accuracy: 0.0517 - val_loss: 0

In [10]:
model.evaluate(Xn_test, yn_test)

752022/752022 [==============================] - 49s 65us/sample - loss: 0.4797 - acc: 0.7856 - categorical_accuracy: 0.1167


[0.4796931303896668, 0.7856206, 0.11665483]